# Pre-procesissing Multi Digit SVHN Dataset: Step 2

- Arman Uygur # au2205
- Jonathan Galsurkar #jfg2150
- Nitesh Surtani #ns3148

In [82]:
"""
Adapted from https://github.com/thomalm/svhn-multi-digit
"""

import os
import sys
import tarfile
import numpy as np
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from IPython.display import display, Image, HTML
import h5py

from skimage import color
from skimage import io

plt.rcParams['figure.figsize'] = (16.0, 4.0)
%matplotlib inline
base_dir = "data/"

In [83]:
import h5py

def readData(d,process):
    h5f = h5py.File(d+process+'_multi_grey.h5','r')
    X = h5f[process+'_dataset'][:]
    y = h5f[process+'_labels'][:]
    h5f.close()
    return X,y
    
X_train, y_train = readData(base_dir,'train')
X_test, y_test = readData(base_dir,'test')
X_extra, y_extra = readData(base_dir,'extra')

In [84]:
X_train_new=color.rgb2gray(X_train)
X_test_new=color.rgb2gray(X_test)
X_extra_new=color.rgb2gray(X_extra)

In [85]:
def random_sample(N, K):
    """Return a boolean mask of size N with K selections
    """
    mask = np.array([True]*K + [False]*(N-K))
    np.random.shuffle(mask)
    return mask

# Pick 3000 training and 2000 extra samples
sample1 = random_sample(X_train_new.shape[0], 3000)
sample2 = random_sample(X_extra_new.shape[0], 20000)

# Create valdidation from the sampled data
X_val_new = np.concatenate([X_train_new[sample1], X_extra_new[sample2]])
y_val = np.concatenate([y_train[sample1], y_extra[sample2]])

# Keep the data not contained by sample
X_train_new = np.concatenate([X_train_new[~sample1], X_extra_new[~sample2]])
y_train = np.concatenate([y_train[~sample1], y_extra[~sample2]])

# Moved to validation and training set

print("Training", X_train_new.shape, y_train.shape)
print('Validation', X_val_new.shape, y_val.shape)

Training (212630, 32, 32) (212630, 5)
Validation (23000, 32, 32) (23000, 5)


In [87]:
# Create file
h5f = h5py.File('data/SVHN_preprocessed.h5', 'w')

# Store the datasets
h5f.create_dataset('train_dataset', data=X_train_new)
h5f.create_dataset('train_labels', data=y_train)
h5f.create_dataset('test_dataset', data=X_test_new)
h5f.create_dataset('test_labels', data=y_test)
h5f.create_dataset('valid_dataset', data=X_val_new)
h5f.create_dataset('valid_labels', data=y_val)

# Close the file
h5f.close()